# Import Libraries 📚

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE, RandomOverSampler
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'pandas'

# Dataset Information ℹ️


In [ ]:
df = pd.read_csv("dataset.csv")
df

# 🔑 Các Đặc Trưng Chính:
Giới tính (GENDER): Giới tính của người tham gia (M: Nam, F: Nữ).

Tuổi (AGE): Tuổi của người tham gia khảo sát.

Hút thuốc (SMOKING): Mức độ hút thuốc (1: Có, 2: Không).

Ngón tay bị vàng (YELLOW_FINGERS): Biểu hiện của việc hút thuốc nhiều.

Lo âu (ANXIETY): Tình trạng lo âu của người được khảo sát.

Áp lực từ bạn bè (PEER_PRESSURE): Mức độ bị ảnh hưởng bởi người xung quanh trong thói quen hút thuốc.

Bệnh mãn tính (CHRONIC_DISEASE): Tình trạng có bệnh nền mãn tính (1: Có, 2: Không).

Mệt mỏi (FATIGUE): Cảm giác mệt mỏi thường xuyên.

Dị ứng (ALLERGY): Có tiền sử bị dị ứng.

Thở khò khè (WHEEZING): Dấu hiệu của các vấn đề hô hấp.

Sử dụng rượu bia (ALCOHOL_CONSUMING): Có uống rượu/bia thường xuyên không.

Ho kéo dài (COUGHING): Có triệu chứng ho mãn tính.

Khó thở (SHORTNESS_OF_BREATH): Triệu chứng thở khó, gấp.

Khó nuốt (SWALLOWING_DIFFICULTY): Gặp khó khăn khi nuốt thức ăn hoặc nước.

Đau ngực (CHEST_PAIN): Triệu chứng đau tức vùng ngực.

# 🎯 Biến Mục Tiêu: Khả năng mắc ung thư phổi (LUNG_CANCER)
YES: Người này có khả năng cao mắc ung thư phổi.

NO: Không có dấu hiệu đáng kể liên quan đến ung thư phổi.

In [ ]:
df.describe().T

In [ ]:
df.info()

In [ ]:
encode = LabelEncoder()
df['GENDER'] = encode.fit_transform(df['GENDER'])
df['Lung_C'] = encode.fit_transform(df['GENDER'])

# Exploratoty Data Analysis (EDA) 📊

In [ ]:
target = df['LUNG_CANCER'].value_counts()
plt.Figure(figsize=(8,8))
plt.pie(target, labels= target.index,autopct='%1.1f%%')
plt.title('Distribution of Label')
plt.show()

<!-- # Training Machine Learning Models 🤖 -->